In [3]:
from __future__ import division, absolute_import, print_function
import sys
if sys.version_info < (3,):
    range = xrange
import os
from pylab import *  # for plotting
from numpy.random import *  # for random sampling
from graph_tool.all import *

In [7]:
#---------------------------------------------Parametros iniciais--------------------------------------------------

#m0: número de sítios iniciais da rede
m0 = 4
#m: número de ligações que os novos nós devem fazer
m = 2 
#t: por quanto tempo o processo será implementando (gerando novos sítios)
t = 10000
#N: número de sítios da rede
N = m0 + t

## Número de sítios da rede

Logo de começo devemos restringir o valor de $m$ com relação a $m_0$, fazendo

<center> $ m \le m_0 $, (1) </center>

essa restrição evita ligações em loop do novo sítio. Nós temos que o número de sítos final da rede será dada por

<center> $N = m_0 + t$, (2) </center>

para cada instante de tempo $t$, haverá $m_0+t$ sítios. Com número de ligações dadas por

<center> $l = m_0+mt$, (3)</center>

pois cada um dos $t$ sítios terão $m$ ligações. Para sabermos quanto devemos utilizar no tempo para um número $N$ de sítios (quantas interações devemos fazer), podemos isolar o $t$ em (2),obtendo

<font size="3"><center> $N-m_0 = t$.  </center></font>

A probabilidade de ligação de um novo sítio $i$ é dada por

<font size="3"><center> $p_i = \frac{k_i}{\sum_i k_i}$ </center></font>,

com a distribuição dos graus dados por

<font size="3"><center> p(k) = $\frac{2m_0(m_0+1)}{k(k+1)(k+2)}$ </center></font>



In [22]:
g = Graph()

#Irei guardar algumas informações de cada sítio e ligação ,para criar alguns mapas de propriedades, para isso
Sitio = g.new_vertex_property("int") #Propriedades dos sítios
Ligacao = g.new_edge_property("int") #Propriedades das sítios

#Tamanho final da rede
N = 100000

#Tamanho inicial da rede
m0 = 5

#Criação da rede para os primeiros nós
v = g.add_vertex()


for i in range(0,m0):
    v = g.add_vertex()
    Sitio[v] = i
    

# we will keep a list of the vertices. The number of times a vertex is in this

# list will give the probability of it being selected.

vlist = [v]


# let's now add the new edges and vertices

for i in range(1, N):

    # create our new vertex

    v = g.add_vertex()

    v_age[v] = i


    # we need to sample a new vertex to be the target, based on its in-degree +

    # 1. For that, we simply randomly sample it from vlist.

    i = randint(0, len(vlist))

    target = vlist[i]


    # add edge

    e = g.add_edge(v, target)

    e_age[e] = i


    # put v and target in the list

    vlist.append(target)

    vlist.append(v)
g.vertex_properties["age"] = v_age

g.edge_properties["age"] = e_age


# now we can save it

g.save("price.xml.gz")


In [23]:
g = load_graph("price.xml.gz")
age = g.vertex_properties["age"]

pos = sfdp_layout(g)
graph_draw(g, pos, output_size=(1000, 1000), vertex_color=[1,1,1,0],
           vertex_fill_color=age, vertex_size=1, edge_pen_width=1.2,
           vcmap=matplotlib.cm.gist_heat_r, output="price.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f7b5b2ffc70, at 0x7f7b5a9c7c10>

## 2. Algoritmo para rede inicial

Tomando a rede inicial com $m_0$ sítios totalmente conectada (sítios igualmente provaveis inicialmente), teremos a matriz de adjacência dada por

\begin{equation}
A = 
\begin{pmatrix}
0 & 1 & 1 & \cdots & 1\\
1 & 0 & 1 & \cdots & 1\\
1 & 1 & 0 & \cdots & 1\\
1 & 1 & 1 & \ddots & 1\\
1 & 1 & 1 & \cdots & 0\\
\end{pmatrix}
_{(m_0,m_0)},
\end{equation}

A rede totalmente conectada, possuí todos elementos fora da coluna principal igual a 1. Para isso, podemos expressar um algoritmo primeiro fixando não-ligação para $i=j$ (sem loops), e logo em seguinda tomando a condição de que $j\le i$, para pegar apenas os elementos do triângulo superior da matriz.

In [84]:
g = Graph(directed=False)

#S = g.new_vertex_property("double")
#L = g.new_edge_property("double")

m0 = 5

vlist = g.add_vertex(m0) #Add m0 elements de sítios em g.

for i in range(m0):
    for j in range(i+1,m0):
        g.add_edge(g.vertex(i),g.vertex(j))
                
print(g.get_out_degrees(g.get_vertices())) #Voltando a rede toda ligada (grau 4 para todos os sítios)
edges = g.get_edges()
u = 0
for i in range(m0):
    u+= g.vertex(i).out_degree()
print(u)

[4 4 4 4 4]
20


In [89]:
a = 2

In [90]:
#m0: número de sítios' iniciais;


def barabasi_network(m0):
    g = Graph(directed=False) #Grafo sem direção
    
    S = g.new_vertex_property("double")
    L = g.new_edge_property("double")
    
    u = 0 #Guardará o valor da soma k_i inicial
    
    v = g.add_vertex()
    vlist = g.add_vertex(m0) #Add m0 elementos de sítios em g.
    
    #Segue o algoritmo da "Rede Inicial", explicado em 2.
        #O loop duplo segue a ideia de fazer todos os sítios ficarem igualmente conectados ((m0-1) ligações)
    
    for i in range(m0):
        for j in range(i+1,m0):
            g.add_edge(g.vertex(i),g.vertex(j)) 
            #Adicionando cada grau ao valor de u, tendo a soma inicial de (m0*(m0-1))
            u += g.vertex(i).out_degree() 
    
    #for i in range(m0,N):
     #   v = g.add_vertex()
      #  S[v] = i
        
        
    
    return u

In [93]:
print(barabasi_network(3))

5
